In [3]:
import json
import numpy as np
res = []
with open('user.json') as f:
    for line in f:
        res.append(json.loads(line))

In [8]:
import pandas as pd
user = pd.DataFrame(res)

In [11]:
res = []
with open('review.json') as f:
    for line in f:
        res.append(json.loads(line))
review = pd.DataFrame(res)

In [17]:
elite_users = user[['user_id', 'elite']][user['elite'] != '']

In [31]:
elite_users = elite_users.reset_index()[['user_id', 'elite']]

In [33]:
elite_users['elite'] = [elite_users['elite'][i].split(',') for i in range(elite_users.shape[0])]

In [57]:
elite_users.head()

,user_id,elite
0,qVc8ODYU5SZjKXVBgXdI7w,[2007]
1,j14WgRoU_-2ZE1aw1dXrJg,"[2009, 2010, 2011, 2012, 2013, 2014, 2015, 201..."
2,2WnXYQFK0hXEoTxPtV2zvg,"[2009, 2010, 2011, 2012, 2013]"
3,SZDeASXq7o05mMNLshsdIA,"[2009, 2010, 2011]"
4,q_QQ5kBBwlCcbL1s4NVK3g,"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201..."


In [52]:
review['date'] = pd.to_datetime(review['date'])

In [90]:
review['year'] = [elem.year for elem in review['date']]

In [92]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,2018
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,2012
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,2014
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,2015
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,2017


In [125]:
import time
start = time.time()
result = []
for i in range(100):
    for j in range(len(elite_users['elite'][i])):
        uid = elite_users['user_id'][i]
        year = int(elite_users['elite'][i][j])
        influencer_review = review.loc[np.logical_and(review['user_id']==uid,review['year']==year),]
        if not influencer_review.empty:
            result.append(influencer_review[['business_id', 'date']])
print(time.time()-start)

3912.899270296097


In [130]:
r = pd.concat(result, axis=0)

In [139]:
r = r.reset_index()[['business_id', 'date']]

In [148]:
r.head()

,business_id,date
0,ompDR5sUDpoI6gnTldmneQ,2010-10-11 19:24:56
1,WtVOrfqLXerig9Z7iP-NzA,2010-10-11 19:49:22
2,DcBLYSvOuWcNReolRVr12A,2010-10-11 17:04:36
3,z_37EXfWEVMV4telxsbGMw,2010-10-11 19:32:30
4,ta5Oi3sezvn4H299MbtE8Q,2010-10-11 20:29:02


In [152]:
r.shape

(4860, 2)

In [206]:
start = time.time()
res = []
for i in range(100):
    bid = r['business_id'][i]
    d = r['date'][i]
    before = review['stars'][np.logical_and(review['business_id']==bid,
                                            d - review['date'] > pd.to_timedelta(0, unit='second'),
                                            d - review['date'] < pd.to_timedelta(30, unit='day'))]
    after = review['stars'][np.logical_and(review['business_id']==bid,
                                          review['date'] - d > pd.to_timedelta(0, unit='second'),
                                          review['date'] - d < pd.to_timedelta(30, unit='day'))]
    res.append([np.mean(after) - np.mean(before), len(after) - len(before)])
print(time.time() - start)

1222.5265669822693


In [203]:
import matplotlib.pyplot as plt
# [elem for elem in res if elem is not np.NAN]
bo = [not elem for elem in pd.isna(res)]
np.array(res)[bo].mean()

-0.04301457324584908

In [1]:
np.nanmean([elem[1] for elem in res])

NameError: name 'np' is not defined